<a href="https://colab.research.google.com/github/cbonnin88/meditrack/blob/main/EDA_and_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import polars as pl
import plotly.express as px
import gdown as gd

In [2]:
url_events = 'https://drive.google.com/uc?id=1DT00olR4KK_zMYXtMEm4q1LViasRc1Gn'
url_subs = 'https://drive.google.com/uc?id=1qNJJ8e8p-VbZ0HkfceOhqxoHsTFGEXGl'
url_users = 'https://drive.google.com/uc?id=1gLZgyIa17KoZsMrwP0-rg6cEFeAoD44O'

In [3]:
gd.download(url_subs,'meditrack_subs_clean.csv',quiet=True)

'meditrack_subs_clean.csv'

In [4]:
gd.download(url_events,'meditrack_events_clean.csv',quiet=True)

'meditrack_events_clean.csv'

In [5]:
gd.download(url_users,'meditrack_users_clean.csv',quiet=True)

'meditrack_users_clean.csv'

In [6]:
df_users = pl.read_csv('meditrack_users_clean.csv')
df_events = pl.read_csv('meditrack_events_clean.csv')
df_subs = pl.read_csv('meditrack_subs_clean.csv')

# **Joining Users and Subscriptions for analysis**

In [7]:
# Using a left join
df_combined = df_users.join(df_subs, on='user_id',how='left')

# **Convert Churned to a string label for better plotting legibility**

In [8]:
df_combined = df_combined.with_columns(
    pl.when(pl.col('churned')== 1).then(pl.lit('Churned'))
    .otherwise(pl.lit('Active'))
    .alias('status')
)

In [9]:
print(f'Data Loaded. Combined Rows: {df_combined.height}')

Data Loaded. Combined Rows: 970


# **Churn by Age**

- **Business Question:** Is our app failing with a specific age group? Are older users dropping off because the font size is too small?

In [11]:
fig_churn_by_age = px.histogram(
    df_combined.to_pandas(),
    x='age',
    color='status',
    barmode='overlay',
    title='User Distribution by Age= Active vs Churned',
    labels={'age':'User Age','count':'Number of Users'},
    opacity=0.7,
    color_discrete_map={'Active':'blue','Churned':'red'}
)

fig_churn_by_age.show()

Daily Active Users (Time Series)

- **Business Question:** Are we growing? Do we have usage spikes on weekends or weekdays?

In [20]:
# Aggregate Events by Date
daily_activity = (
    df_events
    .group_by('event_timestamp')
    .len()
    .rename({'len':'event_count'})
    .sort('event_timestamp')
)

display(daily_activity)

event_timestamp,event_count
str,u32
"""2024-11-27""",16
"""2024-11-28""",21
"""2024-11-29""",17
"""2024-11-30""",22
"""2024-12-01""",14
…,…
"""2025-11-23""",17
"""2025-11-24""",20
"""2025-11-25""",22


In [23]:
fig_daily_active_users = px.line(
    daily_activity.to_pandas(),
    x='event_timestamp',
    y='event_count',
    title='Daily App Events (Growth Metric)',
    markers=True,
    color_discrete_sequence=['#77DD77']
)

fig_daily_active_users.update_layout(xaxis_title='Date',yaxis_title='Total Events')
fig_daily_active_users.show()

# **Revenue Box Plot by Channel**

- **Business Question:** Which marketing channel brings in the "Whales" (users who pay higher fees)?

In [25]:
fig_revenue = px.box(
    df_combined.to_pandas(),
    x='acquisition_channel',
    y='monthly_fee',
    color='acquisition_channel',
    title='Revenue Distribution by Acquisition Channel',
    points='all'
)

fig_revenue.update_layout(xaxis_title='Acquisition Channel',yaxis_title='Monthly Fee')

fig_revenue.show()

# **Activity Heatmap (Day vs Hour)**

- Business Question: When are our users most active? Do we need to scale up servers on Monday mornings or Friday nights?

In [31]:
# Extracting Day and Hour
heatmap_data = (
    df_events
    .with_columns([
        pl.col('event_timestamp').str.to_datetime().dt.weekday().alias('weekday_num'),
        pl.col('event_timestamp').str.to_datetime().dt.hour().alias('hour_of_day')
    ])
    .group_by(['weekday_num','hour_of_day'])
    .len()
    .sort(['weekday_num','hour_of_day'])
)

display(heatmap_data)

weekday_num,hour_of_day,len
i8,i8,u32
1,0,935
2,0,916
3,0,872
4,0,925
5,0,888
6,0,923
7,0,923


In [32]:
# Pivoting for Heatmap format (Matrix) using pandas
heatmap_matrix = heatmap_data.to_pandas().pivot(index='weekday_num',columns='hour_of_day',values='len').fillna(0)

In [37]:
fig_activity = px.imshow(
    heatmap_matrix,
    labels=dict(x='Hour of Day',y='Day of the Week (1=Mon, 7=Sun)', color='Event Volume'),
    x=heatmap_matrix.columns,
    y=heatmap_matrix.index,
    color_continuous_scale='BuPu',
    title='Heatmap: User Activity by Time & Day'
)

fig_activity.show()

# **Engagement vs. Age Scatter Plot**

- Business Question: Do younger people use the app more intensely than older people? Is there a correlation between Age and Total Events?

In [39]:
# Calculate Events per User
user_activity = df_events.group_by('user_id').len().rename({'len': 'total_events'})

display(user_activity)

user_id,total_events
str,u32
"""86052bef-c382-4725-a1c6-3384ee…",4
"""b64a46f5-eb6d-435f-a950-94bbf3…",5
"""13d94ad0-d809-4e6d-b41a-d2c25c…",6
"""25dccca2-054e-4210-a3c9-f400ba…",7
"""1034794e-db2b-41be-ba73-5eb299…",1
…,…
"""77dd362b-4a6b-436b-957f-108b86…",6
"""1c5d9fe9-0b07-4a68-88f2-17a3e2…",7
"""159d880c-76a5-425c-8f90-bfaab4…",5


In [41]:
# Joining with User Demographics
scatter_data = (
    df_users
    .join(user_activity, on='user_id', how='inner')
    .filter(pl.col('age').is_not_null())
)

display(scatter_data)

user_id,signup_date,age,email,country,acquisition_channel,total_events
str,str,i64,str,str,str,u32
"""86052bef-c382-4725-a1c6-3384ee…","""2025-10-23""",38,"""charlesfranco@example.net""","""Denmark""","""Referral""",4
"""b64a46f5-eb6d-435f-a950-94bbf3…","""2025-06-01""",80,"""websterkelly@example.org""","""France""","""Google""",5
"""13d94ad0-d809-4e6d-b41a-d2c25c…","""2025-06-23""",77,"""mjohnson@example.org""","""Denmark""","""Referral""",6
"""25dccca2-054e-4210-a3c9-f400ba…","""2025-11-20""",29,"""donnathomas@example.com""","""Ireland""","""Google""",7
"""1034794e-db2b-41be-ba73-5eb299…","""2025-06-08""",32,"""jeanette06@example.com""","""Ireland""","""Organic""",1
…,…,…,…,…,…,…
"""77dd362b-4a6b-436b-957f-108b86…","""2025-05-06""",65,"""ievans@example.net""","""Ireland""","""Google""",6
"""1c5d9fe9-0b07-4a68-88f2-17a3e2…","""2024-12-14""",47,"""charles59@example.com""","""Denmark""","""Google""",7
"""159d880c-76a5-425c-8f90-bfaab4…","""2025-02-06""",48,"""unknown@meditrack.coom""","""France""","""Referral""",5


In [42]:
fig_engagement = px.scatter(
    scatter_data.to_pandas(),
    x='age',
    y='total_events',
    color='acquisition_channel',
    title='Correlation: User Age vs. App Usage Intensity',
    opacity=0.6,
    trendline='ols'
)

fig_engagement.show()

# **Churn Composition**

- Business Question: Of the people who Churned, what platforms were they on? Did Android users churn more than iOS users?

In [44]:
# Finding the last platform used for every user
last_platform= (
    df_events
    .sort('event_timestamp',descending=True)
    .unique(subset=['user_id'], keep='first')
    .select(['user_id','platform'])
)

display(last_platform)

user_id,platform
str,str
"""7ab417d6-2975-4ef7-9d3f-904a5c…","""Android"""
"""03e7dd21-1d4d-46d6-860d-113d28…","""Android"""
"""1aa6f7cb-90c7-46ac-a3bf-e40338…","""Android"""
"""fef72410-5528-4c45-a37a-a99961…","""Android"""
"""88ccddbc-6f0d-4229-9b38-7c72ac…","""Web"""
…,…
"""0b44f907-6d59-4e12-9ac7-b94d1c…","""Web"""
"""935b67b4-2fbd-4d23-a516-8cda99…","""iOS"""
"""21c4f6d5-95ea-478b-8ce3-edb778…","""iOS"""


In [46]:
# Joining with Subscription Status
churn_platform_data = (
    df_subs
    .join(last_platform, on='user_id')
    .filter(pl.col('churned')==1)
    .group_by('platform')
    .len()
)

display(churn_platform_data)

platform,len
str,u32
"""iOS""",159
"""Web""",177
"""Android""",175


In [51]:
fig_churn = px.pie(
    churn_platform_data.to_pandas(),
    values='len',
    names='platform',
    title='Churn Breakdown by Platform',
    hole=0.4,
    color_discrete_sequence=["#FFB3BA", "#BAFFC9", "#BAE1FF"]
)

fig_churn.show()